In [11]:
# IMPORT LIBRARIES

import numpy as np 
import sympy as sp
import matplotlib.pyplot as plt

In [12]:
# DATA 

T = 60 # Operating temperature [°C]
rho_H2O = 1 # Water density [g/cm3]
NAV = 6.022 * 10 ** 23 # Avogadro number [atoms/mol]
MM_H2O = 18.015 # Water molar mass [g/mol]
rho_235 = 19.1 # Uranium 235 density [g/cm3]
MM_235 = 235.044 # Uranium-235 molar mass [g/mol]
B_Core = 0.05624 # Core Buckling [1/cm]
nu = 2.44 # Neutrons per fission [-]
D_Core = 0.16 # Core diffusion coefficient [cm]
H_core = 50 # Slab width [cm]
q = 25 # Power density [W/cm3]
E_Fission = 211.5 * 10 ** 6  # Energy released by fission [eV]
conversion = 6.242 * 10 ** 18 #Conversion coefficient eV-J
E_Fission = E_Fission / conversion # Energy released by fission [J]

In [13]:
# FUNCTION AND SYMBOLIC VARIABLES DEFINITION

x = sp.symbols('x')
B = sp.symbols('B', nonnegative = True) 
L_c = sp.symbols('L_c', nonnegative = True)
L_r = sp.symbols('L_r', nonnegative = True)
D = sp.symbols('D', nonnegative = True) # D_R = d_C
H = sp.symbols('H', nonnegative = True) # Core width
F_c = sp.Function('phi_c', real = True)(x)
F_r = sp.Function('phi_r', real = True)(x)

In [14]:
############################################################################################################ 
# ODEs DEFINITION
############################################################################################################ 
ODE_c = sp.Eq(sp.diff(sp.diff(F_c, x)) + B** 2 * F_c, 0)
print('ODE FOR THE CORE:')
display(ODE_c)

ODE_r = sp.Eq(sp.diff(sp.diff(F_r, x)) - (1/L_r**2) * F_r, 0)
print('ODE FOR THE REFLECTOR:')
display(ODE_r)

flux_c = sp.dsolve(ODE_c)
sC1 = sp.symbols('C1')   
sC2 = sp.symbols('C2')
print('GENERAL SOLUTION FOR THE FLUX IN THE CORE:')
display(flux_c)

flux_r = sp.dsolve(ODE_r)
sC3 = sp.symbols('C3')
sC4 = sp.symbols('C4')
flux_r = flux_r.subs({sC1: sC3, sC2: sC4})
print('GENERAL SOLUTION FOR THE FLUX IN THE REFLECTOR:')
display(flux_r)


ODE FOR THE CORE:


Eq(B**2*phi_c(x) + Derivative(phi_c(x), (x, 2)), 0)

ODE FOR THE REFLECTOR:


Eq(Derivative(phi_r(x), (x, 2)) - phi_r(x)/L_r**2, 0)

GENERAL SOLUTION FOR THE FLUX IN THE CORE:


Eq(phi_c(x), C1*sin(B*x) + C2*cos(B*x))

GENERAL SOLUTION FOR THE FLUX IN THE REFLECTOR:


Eq(phi_r(x), C3*exp(-x/L_r) + C4*exp(x/L_r))

In [15]:

############################################################################################################ 
# Bounded BC 
############################################################################################################ 
# The flux in the reflector is zero for x that goes to infinity
def apply_boundary_condition(flux, x):
    # Identify the terms in the general solution
    terms = flux.rhs.as_ordered_terms() # Divide the expression into terms
    
    # Determine which term goes to infinity as x -> infinity
    new_terms = []
    for term in terms:
        limit_term = sp.limit(term, x, sp.oo) # Calculate the limit of the term as x -> infinity
        if limit_term.has(sp.oo): # Check if the limit is infinity (any expression that contains oo)
            term = 0 # If the limit is infinity, the term is set to zero
        new_terms.append(term) # Add the term to the new list of terms
            
    flux = sp.Eq(F_r, sum(new_terms)) # Reconstruct the expression with the new terms
    return flux

flux_r = apply_boundary_condition(flux_r, x)
print('BOUNDARY CONDITION FOR THE FLUX IN THE OUTER REGION:')
display(flux_r)


NotImplementedError: Result depends on the sign of -sign(1/L_r)

In [ ]:

############################################################################################################ 
# Continuity BC
############################################################################################################ 
continuity_f = sp.Eq(
    flux_c.rhs.subs(x, H/2), 
    flux_r.rhs.subs(x, H/2)
)
print('CONTINUITY CONDITION FOR THE FLUX AT THE INTERFACE:')
display(continuity_f)
continuity_curr = sp.Eq(
    -D * sp.diff(flux_c.rhs, x).subs(x, H/2),
    -D * sp.diff(flux_r.rhs, x).subs(x, H/2)
    )
print('CONTINUITY CONDITION FOR THE CURRENT AT THE INTERFACE:')
display(continuity_curr)

CONTINUITY CONDITION FOR THE FLUX AT THE INTERFACE:


Eq(C1*sin(B*H/2) + C2*cos(B*H/2), C3)

CONTINUITY CONDITION FOR THE CURRENT AT THE INTERFACE:


Eq(-D*(B*C1*cos(B*H/2) - B*C2*sin(B*H/2)), 0)

In [ ]:
# Create the system of equations
system = [continuity_f, continuity_curr]
print('SYSTEM OF EQUATIONS:')
for eq in system:
    display(eq)

# Solve the system of equations
solution = sp.solve(system, [sC1, sC2, sC3])
print('SOLUTION OF THE SYSTEM OF EQUATIONS:')
display(solution)

# Substitute the solution in the general solution for the core
flux_c = flux_c.subs(solution)
print('FLUX IN THE CORE:')
display(flux_c)


SYSTEM OF EQUATIONS:


Eq(C1*sin(B*H/2) + C2*cos(B*H/2), C3)

Eq(-D*(B*C1*cos(B*H/2) - B*C2*sin(B*H/2)), 0)

SOLUTION OF THE SYSTEM OF EQUATIONS:


{C1: C3*sin(B*H/2)/(sin(B*H/2)**2 + cos(B*H/2)**2),
 C2: C3*cos(B*H/2)/(sin(B*H/2)**2 + cos(B*H/2)**2)}

FLUX IN THE CORE:


Eq(phi_c(x), C3*sin(B*H/2)*sin(B*x)/(sin(B*H/2)**2 + cos(B*H/2)**2) + C3*cos(B*H/2)*cos(B*x)/(sin(B*H/2)**2 + cos(B*H/2)**2))

In [ ]:
# Cross section of the core
Sigma_f = 585 # Macroscopic fission cross section [1/cm]

# Adjust for temperature
Sigma_f = Sigma_f * sp.sqrt(293/(273+T))
print(f'MACROSCOPIC FISSION CROSS SECTION AT T={T}°C:')
display(Sigma_f)

MACROSCOPIC FISSION CROSS SECTION AT T=60°C:


548.741188441229

In [ ]:
# Power condition
power = F_c * E_Fission * Sigma_f
power = power.subs(F_c, flux_c.rhs)
power = power.subs(F_r, flux_r.rhs)
print('POWER CONDITION:')
display(power)

# Integrate the power density over the core
power_integral = sp.integrate(power, (x, 0, H))
display(power_integral)

# Subsitute constants
power_integral = power_integral.subs({B: B_Core, D: D_Core, H: H_core})

# Solve the power integral
power_condition = sp.Eq(power_integral, q*H_core)
sol = sp.solve(power_condition, sC3)
print('C3', sol)


POWER CONDITION:


1.85932011142775e-8*C3*sin(B*H/2)*sin(B*x)/(sin(B*H/2)**2 + cos(B*H/2)**2) + 1.85932011142775e-8*C3*cos(B*H/2)*cos(B*x)/(sin(B*H/2)**2 + cos(B*H/2)**2)

Piecewise((-1.85932011142775e-8*C3*sin(B*H/2)*cos(B*H)/(B*(sin(B*H/2)**2 + cos(B*H/2)**2)) + 1.85932011142775e-8*C3*sin(B*H/2)/(B*(sin(B*H/2)**2 + cos(B*H/2)**2)) + 1.85932011142775e-8*C3*sin(B*H)*cos(B*H/2)/(B*(sin(B*H/2)**2 + cos(B*H/2)**2)), Ne(B, 0)), (1.85932011142775e-8*C3*H*cos(B*H/2)/(sin(B*H/2)**2 + cos(B*H/2)**2), True))

C3 [1916440313.07111]


In [ ]:
# Substitute All the costants into the solutions
C3 = sol[0]
C2 = solution[sC2]
C1 = solution[sC1]
flux_c = flux_c.subs({sC1: C1, sC2: C2, B: B_Core, D: D_Core, H: H_core})
flux_r = flux_r.subs(sC3, C3)

display(flux_c)
display(flux_r)

Eq(phi_c(x), 0.98645178868513*C3*sin(0.05624*x) + 0.164051420597043*C3*cos(0.05624*x))

Eq(phi_r(x), 1916440313.07111)